<a href="https://colab.research.google.com/github/sarahaptumich/capstone/blob/shaptonstall/brace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/Github/capstone/brace

/content/drive/MyDrive/Github/capstone/brace


In [3]:
%ls

annotations/     brace_test.pth   dataset/            lstm_model.pth  utils/
bracedataset.py  brace_train.pkl  dataset_pytorch.py  __pycache__/    videos_info.csv
brace_test.pkl   brace_train.pth  figures/            README.md


In [4]:
# !nvidia-smi

In [5]:
import sys
sys.path.append('/content/drive/MyDrive/Github/capstone/brace')

In [82]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn, optim
import bracedataset
from pathlib import Path
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import TensorDataset
from tqdm import tqdm
from pathlib import PosixPath
import pickle
import re
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split
from torch.optim.lr_scheduler import ExponentialLR
import json
from pathlib import Path

In [7]:
sequences_path = Path('/content/drive/MyDrive/Github/capstone/brace/dataset')
pose_jsons = list(Path(sequences_path).rglob('**/*.json'))
segments= pd.read_csv(Path('/content/drive/MyDrive/Github/capstone/brace/annotations/segments.csv'))

In [9]:
dataClips, labels=bracedataset.BraceDataset(segments, pose_jsons)

In [10]:
unique_categories = list(set(labels))
category_to_int = {category: index for index, category in enumerate(unique_categories)}
encoded_categories = [category_to_int[category] for category in labels]


In [11]:
len(dataClips), type(labels)

(1352, list)

In [13]:
max_length = max(seq.shape[0] for seq in dataClips)


In [23]:
dataClips_tensors=[np.pad(array.astype(np.float32), ((0, max_length - array.shape[0]), (0, 0), (0, 0)), 'constant', constant_values=0) for array in dataClips]
dataClips_tensors=torch.tensor(dataClips_tensors)

<ipython-input-23-742728265591>:2: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  dataClips_tensors=torch.tensor(dataClips_tensors)


In [25]:
label_tensors = np.array(encoded_categories)
label_tensors=torch.tensor(label_tensors)

In [53]:
label_tensors.shape, dataClips_tensors.shape

(torch.Size([1352]), torch.Size([1352, 1280, 17, 2]))

In [70]:
dataset = TensorDataset(dataClips_tensors, label_tensors)
# Set seed for reproducibility
torch.manual_seed(42)

# Determine split sizes
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Split dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

print(f"Training set size: {len(train_dataset)}")
print(f"Testing set size: {len(test_dataset)}")

Training set size: 1081
Testing set size: 271


In [71]:
# # Save the TensorDataset
# with open('tensor_dataset.pkl', 'wb') as f:
#     pickle.dump(dataset, f)

# # Save the Subset
# with open('train_subset.pkl', 'wb') as f:
#     pickle.dump(train_dataset, f)
# with open('test_subset.pkl', 'wb') as f:
#     pickle.dump(test_dataset, f)

In [73]:
batch_size = 16  # Adjust based on your system's capability and the model's requirements

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Use this to load the dataset

In [76]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes=3):  # Default set to 3
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [83]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ExponentialLR(optimizer, gamma=0.9)  # Applies an exponential decay to the learning rate


In [85]:
# Initialize the model
input_size = 17 * 2  # Assuming you flatten the last two dimensions
hidden_size = 128  # Example value
num_layers = 2  # Example value
num_classes = 3  # Adjust based on your task

model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Training step
    for sequences, labels in train_loader:
        sequences = sequences.to(device).view(-1, 1280, 34)  # Adjust shape for LSTM
        labels = labels.to(device)


        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    scheduler.step()  # Update the learning rate after each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, LR: {scheduler.get_last_lr()[0]}')

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Epoch [1/10], Loss: 1.0934, LR: 0.0009000000000000001
Epoch [2/10], Loss: 1.0889, LR: 0.0008100000000000001
Epoch [3/10], Loss: 1.1044, LR: 0.000729


KeyboardInterrupt: 

In [ ]:
# After training loop
model_path = "lstm_model2.pth"
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

In [ ]:
# model = LSTMModel(input_size, hidden_size, num_layers, num_classes).to(device)
# model.load_state_dict(torch.load(model_path))


# print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
correct = 0
total = 0
model.eval()  # Ensure model is in evaluation mode
with torch.no_grad():
    for sequences, labels in test_loader:
        sequences = sequences.to(device).view(-1, 1280, 34)  # Adjust shape for LSTM
        labels = labels.to(device)
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test dataset: {accuracy:.2f}%')